In [1]:
!pip install pandas numpy openpyxl

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path

# Define data directory path
DATA_DIR = Path(r'C:\Users\onurr\ofi-cross-impact-analysis\data')

# First, let's see what files are actually in the directory
print("Files in directory:")
for file in DATA_DIR.glob('*'):
    print(file.name)

Files in directory:
xnas-itch-20241219.mbp-10.csv
xnas-itch-amgn-20241219.mbp-10.csv
xnas-itch-marathon-20241219.mbp-10.csv
xnas-itch-nvdia-20241219.mbp-10.csv
xnas-itch-tsla-20241219.mbp-10.csv


In [5]:
# Define data directory path
DATA_DIR = Path(r'C:\Users\onurr\ofi-cross-impact-analysis\data')

# List of files to process
files = [
    'xnas-itch-20241219.mbp-10.csv',
    'xnas-itch-amgn-20241219.mbp-10.csv',
    'xnas-itch-marathon-20241219.mbp-10.csv',
    'xnas-itch-nvdia-20241219.mbp-10.csv',
    'xnas-itch-tsla-20241219.mbp-10.csv'
]

# Function to examine file structure
def examine_file_structure(file_path):
    try:
        # Read first few rows of the file
        df = pd.read_csv(file_path, nrows=5)
        
        print(f"\nFile: {file_path.name}")
        print("Columns:", df.columns.tolist())
        print("\nFirst row sample:")
        print(df.iloc[0])
        print("\nData types:")
        print(df.dtypes)
        
        return True
    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
        return False

# Process each file
for file in files:
    file_path = DATA_DIR / file
    examine_file_structure(file_path)


File: xnas-itch-20241219.mbp-10.csv
Columns: ['ts_recv', 'ts_event', 'rtype', 'publisher_id', 'instrument_id', 'action', 'side', 'depth', 'price', 'size', 'flags', 'ts_in_delta', 'sequence', 'bid_px_00', 'ask_px_00', 'bid_sz_00', 'ask_sz_00', 'bid_ct_00', 'ask_ct_00', 'bid_px_01', 'ask_px_01', 'bid_sz_01', 'ask_sz_01', 'bid_ct_01', 'ask_ct_01', 'bid_px_02', 'ask_px_02', 'bid_sz_02', 'ask_sz_02', 'bid_ct_02', 'ask_ct_02', 'bid_px_03', 'ask_px_03', 'bid_sz_03', 'ask_sz_03', 'bid_ct_03', 'ask_ct_03', 'bid_px_04', 'ask_px_04', 'bid_sz_04', 'ask_sz_04', 'bid_ct_04', 'ask_ct_04', 'bid_px_05', 'ask_px_05', 'bid_sz_05', 'ask_sz_05', 'bid_ct_05', 'ask_ct_05', 'bid_px_06', 'ask_px_06', 'bid_sz_06', 'ask_sz_06', 'bid_ct_06', 'ask_ct_06', 'bid_px_07', 'ask_px_07', 'bid_sz_07', 'ask_sz_07', 'bid_ct_07', 'ask_ct_07', 'bid_px_08', 'ask_px_08', 'bid_sz_08', 'ask_sz_08', 'bid_ct_08', 'ask_ct_08', 'bid_px_09', 'ask_px_09', 'bid_sz_09', 'ask_sz_09', 'bid_ct_09', 'ask_ct_09', 'symbol']

First row sample:

In [7]:
from pathlib import Path

# Define project root directory
project_root = Path(r'C:\Users\onurr\ofi-cross-impact-analysis')

# Check project structure
required_dirs = ['data', 'notebooks', 'scripts', 'results']

print("Checking project structure...")
for dir_name in required_dirs:
    dir_path = project_root / dir_name
    if dir_path.exists():
        print(f"✓ {dir_name}/ exists")
    else:
        print(f"✗ {dir_name}/ not found")

Checking project structure...
✓ data/ exists
✓ notebooks/ exists
✓ scripts/ exists
✓ results/ exists


In [8]:
# Check if all 5 required data files are in the correct location
from pathlib import Path

# Define data directory path
DATA_DIR = Path(r'C:\Users\onurr\ofi-cross-impact-analysis\data')

# List of required files
required_files = [
    'xnas-itch-20241219.mbp-10.csv',
    'xnas-itch-amgn-20241219.mbp-10.csv',
    'xnas-itch-marathon-20241219.mbp-10.csv',
    'xnas-itch-nvdia-20241219.mbp-10.csv',
    'xnas-itch-tsla-20241219.mbp-10.csv'
]

# Check if data directory exists
if not DATA_DIR.exists():
    print(f"Data directory not found: {DATA_DIR}")
else:
    print("Data directory found")
    # Check each required file
    for file_name in required_files:
        file_path = DATA_DIR / file_name
        if file_path.exists():
            print(f"✓ Found: {file_name}")
        else:
            print(f"✗ Missing: {file_name}")

Data directory found
✓ Found: xnas-itch-20241219.mbp-10.csv
✓ Found: xnas-itch-amgn-20241219.mbp-10.csv
✓ Found: xnas-itch-marathon-20241219.mbp-10.csv
✓ Found: xnas-itch-nvdia-20241219.mbp-10.csv
✓ Found: xnas-itch-tsla-20241219.mbp-10.csv


In [9]:
# Create initial README.md file
readme_content = """# Order Flow Imbalance (OFI) Cross-Impact Analysis

This project analyzes the cross-impact of Order Flow Imbalance (OFI) across multiple stocks.

## Project Structure
- `data/`: Contains the raw market data files
- `notebooks/`: Jupyter notebooks for analysis
- `scripts/`: Python scripts for data processing
- `results/`: Output files and visualizations

## Data Description
Working with high-frequency data for 5 Nasdaq stocks:
- BKR (Baker Hughes)
- NVDA (NVIDIA)
- MARA (Marathon Digital)
- TSLA (Tesla)
"""

with open(project_root / 'README.md', 'w') as f:
    f.write(readme_content)

In [10]:
# Create requirements.txt with necessary packages
requirements_content = """pandas
numpy
matplotlib
seaborn
scikit-learn
jupyter
openpyxl
"""

with open(project_root / 'requirements.txt', 'w') as f:
    f.write(requirements_content)

In [11]:
# Create main analysis script
script_content = """import pandas as pd
import numpy as np
from pathlib import Path

def calculate_ofi(df, level=5):
    '''
    Calculate Order Flow Imbalance for given order book data
    Parameters:
    - df: DataFrame with order book data
    - level: Number of price levels to consider (default=5)
    '''
    ofi_levels = []
    
    for i in range(level):
        # Calculate OFI for each level
        bid_size = df[f'bid_sz_{i:02d}']
        ask_size = df[f'ask_sz_{i:02d}']
        
        # Basic OFI calculation: bid size - ask size
        ofi = bid_size - ask_size
        ofi_levels.append(ofi)
    
    return pd.DataFrame({
        'ts_event': df['ts_event'],
        'symbol': df['symbol'],
        **{f'ofi_level_{i}': ofi_levels[i] for i in range(level)}
    })

# Example usage
if __name__ == "__main__":
    DATA_DIR = Path(__file__).parent.parent / 'data'
    # Process first 1000 rows as a test
    df = pd.read_csv(DATA_DIR / 'xnas-itch-tsla-20241219.mbp-10.csv', nrows=1000)
    ofi_result = calculate_ofi(df)
    print(ofi_result.head())
"""

# Write the script to a file
script_path = project_root / 'scripts' / 'calculate_ofi.py'
with open(script_path, 'w') as f:
    f.write(script_content)

In [12]:
# Create core analysis scripts
scripts = {
    'utils.py': """import pandas as pd
import numpy as np
from pathlib import Path

def load_data(file_path, sample_size=None):
    '''Load and preprocess order book data'''
    df = pd.read_csv(file_path, nrows=sample_size)
    df['ts_event'] = pd.to_datetime(df['ts_event'])
    return df.sort_values('ts_event')

def calculate_ofi(df, level=5):
    '''Calculate OFI metrics for multiple levels'''
    ofi_levels = []
    for i in range(level):
        bid_size = df[f'bid_sz_{i:02d}']
        ask_size = df[f'ask_sz_{i:02d}']
        ofi = bid_size - ask_size
        ofi_levels.append(ofi)
    
    return pd.DataFrame({
        'ts_event': df['ts_event'],
        'symbol': df['symbol'],
        **{f'ofi_level_{i}': ofi_levels[i] for i in range(level)}
    })
""",

    'analysis.py': """import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.decomposition import PCA
from .utils import load_data, calculate_ofi

def process_stock_data(file_path, sample_size=None):
    '''Process single stock data and calculate OFI'''
    df = load_data(file_path, sample_size)
    return calculate_ofi(df)

def calculate_cross_impact(ofi_data, price_changes):
    '''Calculate cross-impact between stocks'''
    return np.corrcoef(ofi_data.T)

def main():
    DATA_DIR = Path(__file__).parent.parent / 'data'
    files = list(DATA_DIR.glob('xnas-itch-*.csv'))
    
    # Process each stock
    all_ofi = {}
    for file in files:
        stock_ofi = process_stock_data(file)
        symbol = stock_ofi['symbol'].iloc[0]
        all_ofi[symbol] = stock_ofi

if __name__ == "__main__":
    main()
""",

    'visualization.py': """import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def plot_ofi_heatmap(cross_impact_matrix, symbols, save_path=None):
    '''Create heatmap of cross-impact relationships'''
    plt.figure(figsize=(10, 8))
    sns.heatmap(cross_impact_matrix, 
                xticklabels=symbols,
                yticklabels=symbols,
                cmap='coolwarm',
                center=0)
    plt.title('Cross-Impact of OFI Between Stocks')
    if save_path:
        plt.savefig(save_path)
    plt.close()
"""
}

# Write all scripts
for filename, content in scripts.items():
    script_path = project_root / 'scripts' / filename
    with open(script_path, 'w') as f:
        f.write(content)

In [13]:
notebook_content = """{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Order Flow Imbalance (OFI) Analysis\\n",
    "This notebook implements the analysis of Order Flow Imbalance cross-impact across multiple stocks."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "import sys\\n",
    "from pathlib import Path\\n",
    "sys.path.append(str(Path.cwd().parent / 'scripts'))\\n",
    "\\n",
    "import pandas as pd\\n",
    "import numpy as np\\n",
    "import matplotlib.pyplot as plt\\n",
    "import seaborn as sns\\n",
    "from utils import load_data, calculate_ofi\\n",
    "from analysis import process_stock_data, calculate_cross_impact\\n",
    "from visualization import plot_ofi_heatmap"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Load and Process Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "DATA_DIR = Path.cwd().parent / 'data'\\n",
    "files = list(DATA_DIR.glob('xnas-itch-*.csv'))\\n",
    "\\n",
    "# Process each stock\\n",
    "all_ofi = {}\\n",
    "for file in files:\\n",
    "    stock_ofi = process_stock_data(file)\\n",
    "    symbol = stock_ofi['symbol'].iloc[0]\\n",
    "    all_ofi[symbol] = stock_ofi\\n",
    "    print(f'Processed {symbol}')"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}
"""

notebook_path = project_root / 'notebooks' / 'ofi_analysis.ipynb'
with open(notebook_path, 'w') as f:
    f.write(notebook_content)

In [16]:
test_script_content = """import pandas as pd
import numpy as np
from pathlib import Path
from utils import load_data, calculate_ofi
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def test_pipeline():
    try:
        # Set paths
        DATA_DIR = Path(__file__).parent.parent / 'data'
        
        # Test with TSLA data first
        test_file = DATA_DIR / 'xnas-itch-tsla-20241219.mbp-10.csv'
        
        # Load first 1000 rows
        logger.info("Loading test data...")
        df = load_data(test_file, sample_size=1000)
        
        # Calculate OFI
        logger.info("Calculating OFI...")
        ofi_results = calculate_ofi(df)
        
        # Basic statistics
        logger.info("\\nOFI Statistics:")
        print(ofi_results.describe())
        
        return True
        
    except Exception as e:
        logger.error(f"Error in test pipeline: {str(e)}")
        return False

if __name__ == "__main__":
    test_pipeline()"""

# Write test script
test_script_path = project_root / 'scripts' / 'test_pipeline.py'
with open(test_script_path, 'w') as f:
    f.write(test_script_content)

In [17]:
# Run test pipeline from scripts directory
from importlib import import_module
import sys

# Add scripts directory to path
scripts_dir = project_root / 'scripts'
if str(scripts_dir) not in sys.path:
    sys.path.append(str(scripts_dir))

# Import and run test
try:
    test_pipeline = import_module('test_pipeline')
    test_pipeline.test_pipeline()
except Exception as e:
    print(f"Error running test pipeline: {e}")

2025-01-05 11:44:19,454 - INFO - Loading test data...
2025-01-05 11:44:19,489 - INFO - Calculating OFI...
2025-01-05 11:44:19,491 - INFO - 
OFI Statistics:
       ofi_level_0  ofi_level_1  ofi_level_2  ofi_level_3  ofi_level_4
count  1000.000000  1000.000000  1000.000000  1000.000000  1000.000000
mean     12.771000   -39.029000  -184.686000  -158.945000  -169.527000
std     173.487932   224.798213   230.994753   225.855522   248.655989
min    -499.000000  -499.000000  -499.000000  -516.000000  -516.000000
25%     -66.000000  -165.000000  -450.000000  -408.000000  -474.000000
50%     -16.000000    -4.500000  -165.000000  -100.000000  -124.000000
75%      90.000000    80.000000   -10.000000    14.000000     9.000000
max     450.000000   409.000000   359.000000   358.000000   700.000000


In [18]:
full_analysis_content = """import pandas as pd
import numpy as np
from pathlib import Path
from utils import load_data, calculate_ofi
from visualization import plot_ofi_heatmap
import logging
from datetime import datetime

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def run_full_analysis():
    try:
        # Set paths
        DATA_DIR = Path(__file__).parent.parent / 'data'
        RESULTS_DIR = Path(__file__).parent.parent / 'results'
        RESULTS_DIR.mkdir(exist_ok=True)
        
        # Process all files
        all_stocks_ofi = {}
        files = list(DATA_DIR.glob('xnas-itch-*.csv'))
        
        for file in files:
            logger.info(f"Processing {file.name}")
            df = load_data(file)
            ofi_results = calculate_ofi(df)
            symbol = ofi_results['symbol'].iloc[0]
            all_stocks_ofi[symbol] = ofi_results
            
            # Save individual stock results
            output_file = RESULTS_DIR / f'ofi_results_{symbol}.csv'
            ofi_results.to_csv(output_file, index=False)
            logger.info(f"Saved results for {symbol}")
            
        # Calculate cross-impacts
        logger.info("Calculating cross-impacts...")
        symbols = list(all_stocks_ofi.keys())
        cross_impact_matrix = np.zeros((len(symbols), len(symbols)))
        
        for i, sym1 in enumerate(symbols):
            for j, sym2 in enumerate(symbols):
                corr = all_stocks_ofi[sym1]['ofi_level_0'].corr(all_stocks_ofi[sym2]['ofi_level_0'])
                cross_impact_matrix[i, j] = corr
        
        # Create and save heatmap
        plot_ofi_heatmap(cross_impact_matrix, symbols, 
                        save_path=RESULTS_DIR / 'cross_impact_heatmap.png')
        
        # Save cross-impact matrix
        pd.DataFrame(cross_impact_matrix, 
                    index=symbols, 
                    columns=symbols).to_csv(RESULTS_DIR / 'cross_impact_matrix.csv')
        
        logger.info("Analysis completed successfully")
        return True
        
    except Exception as e:
        logger.error(f"Error in full analysis: {str(e)}")
        return False

if __name__ == "__main__":
    run_full_analysis()
"""

# Write full analysis script
analysis_path = project_root / 'scripts' / 'full_analysis.py'
with open(analysis_path, 'w') as f:
    f.write(full_analysis_content)

In [19]:
# Run full analysis
try:
    import full_analysis
    full_analysis.run_full_analysis()
except Exception as e:
    print(f"Error running full analysis: {e}")

2025-01-05 11:45:07,254 - INFO - Processing xnas-itch-20241219.mbp-10.csv
2025-01-05 11:45:11,045 - INFO - Saved results for BKR
2025-01-05 11:45:11,046 - INFO - Processing xnas-itch-amgn-20241219.mbp-10.csv
2025-01-05 11:46:29,292 - INFO - Saved results for NVDA
2025-01-05 11:46:29,307 - INFO - Processing xnas-itch-marathon-20241219.mbp-10.csv
2025-01-05 11:46:36,665 - INFO - Saved results for MARA
2025-01-05 11:46:36,665 - INFO - Processing xnas-itch-nvdia-20241219.mbp-10.csv
2025-01-05 11:47:44,302 - INFO - Saved results for NVDA
2025-01-05 11:47:44,309 - INFO - Processing xnas-itch-tsla-20241219.mbp-10.csv
2025-01-05 11:48:17,515 - INFO - Saved results for TSLA
2025-01-05 11:48:17,519 - INFO - Calculating cross-impacts...
2025-01-05 11:48:19,587 - INFO - Analysis completed successfully


In [20]:
# Check all generated files
results_dir = project_root / 'results'
print("Generated Results Files:")
for file in results_dir.glob('*'):
    print(f"- {file.name}")

Generated Results Files:
- .gitkeep
- cross_impact_heatmap.png
- cross_impact_matrix.csv
- ofi_results_BKR.csv
- ofi_results_MARA.csv
- ofi_results_NVDA.csv
- ofi_results_TSLA.csv


In [21]:
# Read and display the cross-impact matrix
cross_impact = pd.read_csv(results_dir / 'cross_impact_matrix.csv', index_col=0)
print("Cross-Impact Matrix:")
print(cross_impact)

# Check one of the individual OFI results (let's look at TSLA)
tsla_ofi = pd.read_csv(results_dir / 'ofi_results_TSLA.csv')
print("\nTSLA OFI Results (first few rows):")
print(tsla_ofi.head())

# Get basic statistics of OFI for each stock
print("\nBasic OFI Statistics for each stock:")
for file in results_dir.glob('ofi_results_*.csv'):
    df = pd.read_csv(file)
    symbol = df['symbol'].iloc[0]
    print(f"\n{symbol}:")
    print(df['ofi_level_0'].describe())

Cross-Impact Matrix:
           BKR      NVDA      MARA      TSLA
BKR   1.000000  0.009360 -0.026643  0.005796
NVDA  0.009360  1.000000 -0.013308  0.046677
MARA -0.026643 -0.013308  1.000000 -0.027286
TSLA  0.005796  0.046677 -0.027286  1.000000

TSLA OFI Results (first few rows):
                              ts_event symbol  ofi_level_0  ofi_level_1  \
0  2024-12-19 09:00:00.011910681+00:00   TSLA          300            0   
1  2024-12-19 09:00:00.012234472+00:00   TSLA          300          300   
2  2024-12-19 09:00:00.014157632+00:00   TSLA          200          300   
3  2024-12-19 09:00:00.014191599+00:00   TSLA          200          200   
4  2024-12-19 09:00:00.017483329+00:00   TSLA          200          200   

   ofi_level_2  ofi_level_3  ofi_level_4  
0            0            0            0  
1            0            0            0  
2          300            0            0  
3          300          300            0  
4          300          300          700  

Basic OF

In [22]:
import matplotlib.pyplot as plt
import seaborn as sns

# Create heatmap of cross-impact matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cross_impact, 
            annot=True, 
            cmap='coolwarm', 
            center=0,
            fmt='.4f')
plt.title('Cross-Impact Matrix Heatmap')
plt.savefig(results_dir / 'cross_impact_detailed_heatmap.png')
plt.close()

# Create comparison of OFI distributions
fig, ax = plt.subplots(2, 2, figsize=(15, 15))
stocks = ['BKR', 'NVDA', 'MARA', 'TSLA']

for idx, stock in enumerate(stocks):
    i, j = divmod(idx, 2)
    df = pd.read_csv(results_dir / f'ofi_results_{stock}.csv')
    sns.histplot(data=df, x='ofi_level_0', bins=50, ax=ax[i,j])
    ax[i,j].set_title(f'{stock} OFI Distribution')
    ax[i,j].set_xlabel('OFI Level 0')

plt.tight_layout()
plt.savefig(results_dir / 'ofi_distributions.png')
plt.close()

In [23]:
# Check if new visualization files were created
print("Files in results directory:")
for file in results_dir.glob('*.png'):
    print(f"- {file.name}")

Files in results directory:
- cross_impact_detailed_heatmap.png
- cross_impact_heatmap.png
- ofi_distributions.png


In [27]:
from sklearn.decomposition import PCA
import datetime as dt

def enhance_analysis():
    """Add PCA and lagged analysis to our existing results"""
    results_dir = project_root / 'results'
    stocks = ['BKR', 'NVDA', 'MARA', 'TSLA']
    
    # 1. PCA Analysis
    all_ofi_levels = {}
    for stock in stocks:
        df = pd.read_csv(results_dir / f'ofi_results_{stock}.csv')
        ofi_columns = [col for col in df.columns if 'ofi_level' in col]
        ofi_matrix = df[ofi_columns].values
        
        # Apply PCA
        pca = PCA()
        pca_result = pca.fit_transform(ofi_matrix)
        
        # Save PCA results
        pca_df = pd.DataFrame(
            pca_result[:, :2],  # Keep first two components
            columns=['PC1', 'PC2']
        )
        pca_df['symbol'] = stock
        pca_df['ts_event'] = pd.to_datetime(df['ts_event'])
        pca_df.to_csv(results_dir / f'pca_results_{stock}.csv', index=False)
        
        # Save explained variance
        explained_var = pd.DataFrame({
            'component': range(1, len(pca.explained_variance_ratio_) + 1),
            'explained_variance_ratio': pca.explained_variance_ratio_
        })
        explained_var.to_csv(results_dir / f'explained_variance_{stock}.csv', index=False)
    
    # 2. Lagged Analysis (1-min and 5-min)
    lag_periods = [1, 5]  # minutes
    for stock in stocks:
        df = pd.read_csv(results_dir / f'ofi_results_{stock}.csv')
        df['ts_event'] = pd.to_datetime(df['ts_event'])
        
        # Resample to 1-minute intervals
        df.set_index('ts_event', inplace=True)
        resampled = df['ofi_level_0'].resample('1T').mean()
        
        # Calculate lagged correlations
        lag_results = []
        for lag in lag_periods:
            corr = resampled.corr(resampled.shift(lag))
            lag_results.append({
                'stock': stock,
                'lag_minutes': lag,
                'autocorrelation': corr
            })
        
        # Save lag analysis
        lag_df = pd.DataFrame(lag_results)
        lag_df.to_csv(results_dir / f'lag_analysis_{stock}.csv', index=False)

    print("Enhanced analysis completed")
    
# Run enhanced analysis
enhance_analysis()

Enhanced analysis completed


In [28]:
# Enhanced visualizations
import seaborn as sns
from matplotlib.gridspec import GridSpec
import matplotlib.dates as mdates

# 1. Time Series Analysis of OFI
def create_time_series_plot():
    plt.figure(figsize=(15, 10))
    for stock in ['BKR', 'NVDA', 'MARA', 'TSLA']:
        df = pd.read_csv(results_dir / f'ofi_results_{stock}.csv')
        df['ts_event'] = pd.to_datetime(df['ts_event'])
        # Resample to 5-minute intervals for better visualization
        df.set_index('ts_event', inplace=True)
        ofi_resampled = df['ofi_level_0'].resample('5T').mean()
        plt.plot(ofi_resampled.index, ofi_resampled.values, label=stock)
    
    plt.title('OFI Time Series (5-minute intervals)')
    plt.xlabel('Time')
    plt.ylabel('OFI Level 0')
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(results_dir / 'ofi_time_series.png')
    plt.close()

# 2. Multi-level OFI Analysis
def create_multilevel_comparison():
    fig = plt.figure(figsize=(15, 10))
    gs = GridSpec(2, 2, figure=fig)
    
    for idx, stock in enumerate(['BKR', 'NVDA', 'MARA', 'TSLA']):
        i, j = divmod(idx, 2)
        ax = fig.add_subplot(gs[i, j])
        
        df = pd.read_csv(results_dir / f'ofi_results_{stock}.csv')
        for level in range(5):
            sns.kdeplot(data=df[f'ofi_level_{level}'], label=f'Level {level}', ax=ax)
        
        ax.set_title(f'{stock} OFI Distribution Across Levels')
        ax.set_xlabel('OFI Value')
        ax.legend()
    
    plt.tight_layout()
    plt.savefig(results_dir / 'multilevel_ofi_comparison.png')
    plt.close()

# 3. Box Plot Analysis
def create_box_plots():
    plt.figure(figsize=(15, 8))
    box_data = []
    labels = []
    for stock in ['BKR', 'NVDA', 'MARA', 'TSLA']:
        df = pd.read_csv(results_dir / f'ofi_results_{stock}.csv')
        box_data.append(df['ofi_level_0'])
        labels.append(stock)
    
    plt.boxplot(box_data, labels=labels, showfliers=False)
    plt.title('OFI Distribution Comparison')
    plt.ylabel('OFI Level 0')
    plt.grid(True, alpha=0.3)
    plt.savefig(results_dir / 'ofi_boxplot_comparison.png')
    plt.close()

# Create all visualizations
try:
    create_time_series_plot()
    create_multilevel_comparison()
    create_box_plots()
    print("Additional visualizations created successfully")
except Exception as e:
    print(f"Error creating visualizations: {e}")

Additional visualizations created successfully
